In [ ]:
Stock Analysis Report

In [ ]:
import os
import pandas as pd
import yfinance as yf
from dotenv import load_dotenv
from sqlalchemy import create_engine


In [ ]:
#Top 10 Tech Stocks tickers
TICKERS = ['NVDA', 'MSFT', 'AAPL', 'GOOGL', 'AMZN', 'META', 'AVGO', 'TSLA', 'TSM', 'ORCL']

#Load variables from .env file
load_dotenv()

#Extract Data from yfinance
def extract_and_load_stocks():

    print(f"Fetching data for: {', '.join(TICKERS)}...")

        try:
            stock_data = yf.download(tickers, start="2021-01-01", end="2026-01-01", interval="1d")

            if stock_data.empty:
                print("Error: No data fetched. Check your internet connection or tickers.")
                return
                # 3. Transform yfinance 
            adj_close = stock_data['Adj Close'].melt(ignore_index=False, var_name='Ticker', value_name='Price')
            volume = stock_data['Volume'].melt(ignore_index=False, var_name='Ticker', value_name='Volume')

            df = adj_close.copy()
            df['Volume'] = volume['Volume']
            df = df.reset_index()
        
            # 2. Remove any rows where price might be NaN
            df.dropna(subset=['Price'], inplace=True)

            # 4. Connect to PostgreSQL
            user = os.getenv('DB_USER')
            password = os.getenv('DB_PASSWORD')
            host = os.getenv('DB_HOST')
            port = os.getenv('DB_PORT')
            dbname = os.getenv('DB_NAME')

            connection_string = f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
            engine = create_engine(connection_string)

            print("Uploading data to PostgreSQL...")
        
            # 3. Upload data to SQL
            df.to_sql('raw_stocks', engine, if_exists='replace', index=False, method='multi')
        
            print("Success!")

        except Exception as e:
            print(f"An error occurred: {e}")

if __name__ == "__main__":
    extract_and_load_stocks()